In [ ]:
import pandas as pd

df = pd.read_csv('sample_data/100QA.csv')
df.head()

In [ ]:
def tokenize(text):
    text = text.lower()
    text = text.replace("?", "")
    text = text.replace("'", "")
    return text.split()


In [ ]:
vocab = {"<UNK>": 0}
def build_vocab(row):
    q = tokenize(row.question)
    a = tokenize(row.answer)
    for word in q + a:
        if word not in vocab:
            vocab[word] = len(vocab)

df.apply(build_vocab, axis='columns')
# axis = 1 is also ok

vocab

In [ ]:
def text_to_indices(text, vocab):
    indices = []
    for word in tokenize(text):
        if word in vocab:
            indices.append(vocab[word])
        else:
            indices.append(vocab["<UNK>"])
    return indices

text_to_indices(df.question[1], vocab)

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset

class QADataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        q = text_to_indices(row.question, self.vocab)
        a = text_to_indices(row.answer, self.vocab)
        return torch.tensor(q), torch.tensor(a)

In [ ]:
dataset = QADataset(df, vocab)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

for q, a in dataloader:
    print(q, a)

In [ ]:
# class SimpleRNN(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.embedding = nn.Embedding(vocab_size, 50)
#         self.RNN = nn.RNN(50, 64)
#         self.fc = nn.Linear(64, vocab_size)

#     def forward(self, x):
#         pass

# x = nn.Embedding(324, 50)
# # dataset[0][0], x(dataset[0][0]).shape
# a = x(dataset[0][0])
# y = nn.RNN(50, 64)
# # y(a)[0].shape, y(a)[1].shape, y(a)[0], y(a)[1]
# # pehle vale ne 6 vectors diye (O(1), O(2), ... O(6)) aur dusre ne 1 diya (O(6))
# b = y(a)[1]
# z = nn.Linear(64, 324)
# # z(b).shape, z(b)

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 50)
        self.rnn = nn.RNN(50, 64, batch_first=True)
        self.fc = nn.Linear(64, vocab_size) # fc = fully connected layer
        # yahan pe nn.Sequential (jo sirf ek hi output ko pass on karna allow karta hai) use nhi
        # kar sakte kyuki hame 2 outputs mil rhe hain jo ki dono ham next layer me bhej rhe hain

    def forward(self, question):
        hidden, final = self.rnn(self.embedding(question))
        return self.fc(final.squeeze(0))

In [ ]:
learning_rate = 0.001
epochs = 50

model = SimpleRNN(len(vocab))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
x = nn.Embedding(324, 50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)
a = dataset[0][0].reshape(1, 6)
b = x(a)
c, d = y(b)
e = z(d.squeeze(0))
a.shape, b.shape, c.shape, d.shape, e.shape

In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for q, a in dataloader:
        optimizer.zero_grad()
        loss = loss_fn(model(q), a[0])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

In [ ]:
def predict(model, question, threshold=0.5):
    question_tensor = torch.tensor(text_to_indices(question, vocab)).unsqueeze(0)
    output = model(question_tensor)
    probs = nn.functional.softmax(output, dim=1)
    value, index = torch.max(probs, dim=1)
    if value < threshold:
        print("I don't know", value)
    else:
        print(list(vocab.keys())[index], value)

predict(model, "what is the largest planet in our solar system")